# 顧客流失預警分析（數位訂閱產業案例&存活分析）

## 目的
1. 利用「時間序列」相關之機器學習方法從多種變數中挑選出具有影響的變數
2. 得知每一位客戶重要調整的變數為何，讓行銷人員更容易根據數據下手調整行銷策略
3. 藉由行銷策略的調整（欄位調整），試圖增加顧客終身價值
4. 將資料集時間因素考慮入模型，可以快速知道每一位會員每一個時間點存留的機率

In [1]:
import numpy as np
import pandas as pd
from util import get_dummies, detect_str_columns, model_testRF

# 存活分析的套件：流失預警分析的套件
from lifelines import CoxPHFitter

## 一、資料前處理

In [2]:
customer_data = pd.read_csv('Customer_member_censored.csv',encoding = 'cp950')
customer_data.head()

,customerID,gender,長輩服務方案,伴侶服務方案,親子服務方案,已存留時間,瀏覽過A_landing_page,coupon資訊,A_Channel獨享優惠,行銷_九折優惠,...,產品搭售B資訊,EDM,加值功能A,加值功能B,合約年限,加值功能C,付款方式,目前月費,目前累計總費用,存留狀況
0,1120_QHQEC,Female,0,Yes,No,1,No,No,dis_A,No,...,No,No,No,No,Month-to-month,Yes,electronic_auto,29.85,29.85,No
1,1111_CNQDE,Male,0,No,No,34,Yes,No,dis_A,Yes,...,Yes,No,No,No,One year,No,mail,56.95,1889.5,No
2,4668_QPYBI,Male,0,No,No,2,Yes,No,dis_A,Yes,...,No,No,No,No,Month-to-month,Yes,mail,53.85,108.15,Yes
3,1121_CFTCW,Male,0,No,No,45,No,No,dis_A,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,2241_HQITU,Female,0,No,No,2,Yes,No,dis_B,No,...,No,No,No,No,Month-to-month,Yes,electronic_auto,70.70,151.65,Yes


**將customerID轉換為index（set_index），這樣customerID就不會放入分析欄位中**

In [3]:
# 以往我們要將UID刪除，用del or drop
# 我可以將UID設定成「index」，那機器學習時候就不會用到它，
# 之後要使用的時候，再叫出來
customer_data = customer_data.set_index('customerID')
customer_data.head()

,gender,長輩服務方案,伴侶服務方案,親子服務方案,已存留時間,瀏覽過A_landing_page,coupon資訊,A_Channel獨享優惠,行銷_九折優惠,產品搭售A資訊,產品搭售B資訊,EDM,加值功能A,加值功能B,合約年限,加值功能C,付款方式,目前月費,目前累計總費用,存留狀況
customerID,,,,,,,,,,,,,,,,,,,,
1120_QHQEC,Female,0,Yes,No,1,No,No,dis_A,No,Yes,No,No,No,No,Month-to-month,Yes,electronic_auto,29.85,29.85,No
1111_CNQDE,Male,0,No,No,34,Yes,No,dis_A,Yes,No,Yes,No,No,No,One year,No,mail,56.95,1889.5,No
4668_QPYBI,Male,0,No,No,2,Yes,No,dis_A,Yes,Yes,No,No,No,No,Month-to-month,Yes,mail,53.85,108.15,Yes
1121_CFTCW,Male,0,No,No,45,No,No,dis_A,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
2241_HQITU,Female,0,No,No,2,Yes,No,dis_B,No,No,No,No,No,No,Month-to-month,Yes,electronic_auto,70.70,151.65,Yes


**【目前累計總費用】因為是最後的產物（已經成了結果），無法進行後續的調整，所以在分析的時候暫時不考慮**

In [4]:
customer_data = customer_data.drop( columns = ['目前累計總費用'])

**轉換虛擬變數</font>**<br>
&emsp;1. 偵測有字串的欄位
&emsp;2. 挑選出來，準備encoding

In [5]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 1120_QHQEC to 4186_AJIEI
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             7043 non-null   object 
 1   長輩服務方案             7043 non-null   int64  
 2   伴侶服務方案             7043 non-null   object 
 3   親子服務方案             7043 non-null   object 
 4   已存留時間              7043 non-null   int64  
 5   瀏覽過A_landing_page  7043 non-null   object 
 6   coupon資訊           7043 non-null   object 
 7   A_Channel獨享優惠      7043 non-null   object 
 8   行銷_九折優惠            7043 non-null   object 
 9   產品搭售A資訊            7043 non-null   object 
 10  產品搭售B資訊            7043 non-null   object 
 11  EDM                7043 non-null   object 
 12  加值功能A              7043 non-null   object 
 13  加值功能B              7043 non-null   object 
 14  合約年限               7043 non-null   object 
 15  加值功能C              7043 non-null   object 
 16  付款方式          

In [6]:
str_columns = detect_str_columns(customer_data)
customer_data = get_dummies(str_columns, customer_data)
customer_data.head()

,長輩服務方案,已存留時間,目前月費,gender_Male,伴侶服務方案_Yes,親子服務方案_Yes,瀏覽過A_landing_page_Yes,coupon資訊_Yes,A_Channel獨享優惠_dis_A,A_Channel獨享優惠_dis_B,...,EDM_Yes,加值功能A_Yes,加值功能B_Yes,合約年限_One year,合約年限_Two year,加值功能C_Yes,付款方式_Credit card (automatic),付款方式_electronic_auto,付款方式_mail,存留狀況_Yes
customerID,,,,,,,,,,,,,,,,,,,,,
1120_QHQEC,0,1,29.85,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
1111_CNQDE,0,34,56.95,1,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,1,0
4668_QPYBI,0,2,53.85,1,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,1,1
1121_CFTCW,0,45,42.30,1,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0
2241_HQITU,0,2,70.70,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,1,0,1


**將data欄位轉換成「數值」後</font>**<br>
&emsp;1. 用眼睛看一下data frame
&emsp;2. 用info看欄位是否為int, float</font>

In [7]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 1120_QHQEC to 4186_AJIEI
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   長輩服務方案                        7043 non-null   int64  
 1   已存留時間                         7043 non-null   int64  
 2   目前月費                          7043 non-null   float64
 3   gender_Male                   7043 non-null   uint8  
 4   伴侶服務方案_Yes                    7043 non-null   uint8  
 5   親子服務方案_Yes                    7043 non-null   uint8  
 6   瀏覽過A_landing_page_Yes         7043 non-null   uint8  
 7   coupon資訊_Yes                  7043 non-null   uint8  
 8   A_Channel獨享優惠_dis_A           7043 non-null   uint8  
 9   A_Channel獨享優惠_dis_B           7043 non-null   uint8  
 10  行銷_九折優惠_Yes                   7043 non-null   uint8  
 11  產品搭售A資訊_Yes                   7043 non-null   uint8  
 12  產品搭售B資訊_Yes                   7043 non-null   uint8 

## 二、建立存活分析模組

**存出初始模型</font>**

In [8]:
# 命名模型物件
# 同這種概念：xgb_model = XGBClassifier( n_estimator = 100 )
cph = CoxPHFitter()

**使用初始模型建模</font>**<br>
**注意：存留狀況_Yes --> 代表「流失」</font>**<br>
**format: cph.fit (data, time, churn/out/流失)</font>**

In [9]:
# how to get 已存留時間：
# 1.知道產業特性：知道多久的資料還可以用？ 本案是72月內皆可以用
# 2.訂閱的形式的時間：是月、周、日? 本案是月，所以「已存留時間」以月定之

# 存留狀況：
# 1.共72月的資料，裡面的人員至今是否存留。舉例：A人目前存留了24個月，離開了，會顯示1（流失），在存活分析裡面會以「uncensored/deceased」為代表; 
# 沒有流失的話，會以censored為代表
cph.fit(customer_data, '已存留時間', '存留狀況_Yes')

# 本案共有5174是存在；7043 - 5174是流失

<lifelines.CoxPHFitter: fitted with 7043 total observations, 5174 right-censored observations>

In [10]:
customer_data['存留狀況_Yes'].value_counts()

0    5174
1    1869
Name: 存留狀況_Yes, dtype: int64

## 三、建立具有影響的變數

In [13]:
from util import survival_coef
keep_coef2 = survival_coef(cph)

Index(['變數', 'coef', 'exp(coef)', 'se(coef)', 'coef lower 95%',
       'coef upper 95%', 'exp(coef) lower 95%', 'exp(coef) upper 95%',
       'cmp to', 'z', 'p', '-log2(p)', '增減量%', '增減量下限信賴區間%', '增減量上限信賴區間%',
       '迴歸參數解釋'],
      dtype='object')


c:\Users\saman\OneDrive\桌面\TMR\課程\機器學習實作\day3_課後\機器學習實作案例\2_顧客流失預警分析\2_顧客流失預警\util.py:78: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\saman\OneDrive\桌面\TMR\課程\機器學習實作\day3_課後\機器學習實作案例\2_顧客流失預警分析\2_顧客流失預警\util.py:79: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\saman\OneDrive\桌面\TMR\課程\機器學習實作\day3_課後\機器學習實作案例\2_顧客流失預警分析\2_顧客流失預警\util.py:82: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [14]:
keep_coef2

,變數,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p),增減量%,增減量下限信賴區間%,增減量上限信賴區間%,迴歸參數解釋
16,合約年限_Two year,-3.23,0.04,0.16,-3.56,-2.91,0.03,0.05,0.0,-19.60,0.00,281.78,-0.96,-0.97,-0.95,減少96.0%流失的可能性
15,合約年限_One year,-1.62,0.20,0.09,-1.79,-1.44,0.17,0.24,0.0,-18.30,0.00,246.19,-0.80,-0.83,-0.76,減少80.0%流失的可能性
9,行銷_九折優惠_Yes,-0.61,0.54,0.13,-0.87,-0.35,0.42,0.70,0.0,-4.60,0.00,17.84,-0.46,-0.58,-0.30,減少46.0%流失的可能性
10,產品搭售A資訊_Yes,-0.61,0.54,0.13,-0.86,-0.36,0.42,0.69,0.0,-4.83,0.00,19.48,-0.46,-0.58,-0.31,減少46.0%流失的可能性
3,伴侶服務方案_Yes,-0.52,0.60,0.06,-0.63,-0.41,0.53,0.66,0.0,-9.40,0.00,67.30,-0.40,-0.47,-0.34,減少40.0%流失的可能性
6,coupon資訊_Yes,-0.42,0.66,0.13,-0.67,-0.17,0.51,0.84,0.0,-3.31,0.00,10.07,-0.34,-0.49,-0.16,減少34.0%流失的可能性
12,EDM_Yes,-0.37,0.69,0.13,-0.63,-0.11,0.53,0.90,0.0,-2.79,0.01,7.58,-0.31,-0.47,-0.10,減少31.0%流失的可能性
11,產品搭售B資訊_Yes,-0.28,0.75,0.13,-0.53,-0.04,0.59,0.96,0.0,-2.26,0.02,5.38,-0.25,-0.41,-0.04,減少25.0%流失的可能性
17,加值功能C_Yes,0.18,1.20,0.06,0.07,0.29,1.07,1.34,0.0,3.20,0.00,9.52,0.20,0.07,0.34,增加20.0%流失的可能性
20,付款方式_mail,0.57,1.76,0.09,0.39,0.74,1.48,2.09,0.0,6.42,0.00,32.73,0.76,0.48,1.09,增加76.0%流失的可能性


## 四、廣告效益圖 - 繪製95%信賴區間
讓變數`效應`一覽無遺

In [15]:
# -----可視化分析成果－繪圖-----
from util import campaign_analysis
campaign_analysis(keep_coef2, '廣告效益圖 - 繪製95%信賴區間')

c:\xubwei\Anaconda\lib\site-packages\plotly\offline\offline.py:562: UserWarning:

Your filename `廣告效益圖 - 繪製95%信賴區間` didn't end with .html. Adding .html to the end of your file.



## 五、找出顧客終身價值

In [17]:
from util import customer_marketng_strategy_func
value = customer_marketng_strategy_func(
        # data
        customer_data=customer_data,

        # 我想要做行銷調整的客戶
        customer_test_name = '1120_QHQEC',

        # 模型
        cph = cph,

        #節省行銷預算的調整表
        keep_coef2 = keep_coef2
                                )

c:\Users\saman\OneDrive\桌面\TMR\課程\機器學習實作\day3_課後\機器學習實作案例\2_顧客流失預警分析\2_顧客流失預警\util.py:207: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\saman\OneDrive\桌面\TMR\課程\機器學習實作\day3_課後\機器學習實作案例\2_顧客流失預警分析\2_顧客流失預警\util.py:208: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\saman\OneDrive\桌面\TMR\課程\機器學習實作\day3_課後\機器學習實作案例\2_顧客流失預警分析\2_顧客流失預警\util.py:209: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [18]:
value

,customerID,變數,id,增減量%,後續行銷策略,個體預測存留月數,個體可增加存留月數,個體轉變後存留月數,目前月費,顧客終身價值
3,1120_QHQEC,合約年限_Two year,0.0,-0.96,推薦,36.974946,32.547358,69.522304,29.85,2075.240776
4,1120_QHQEC,合約年限_One year,0.0,-0.80,推薦,36.974946,24.032582,61.007528,29.85,1821.074698
2,1120_QHQEC,A_Channel獨享優惠_dis_A,1.0,2.67,轉換,36.974946,20.751497,57.726443,29.85,1723.134322
5,1120_QHQEC,行銷_九折優惠_Yes,0.0,-0.46,推薦,36.974946,11.074932,48.049878,29.85,1434.288863
1,1120_QHQEC,付款方式_electronic_auto,1.0,0.80,轉換,36.974946,10.640147,47.615093,29.85,1421.310526
6,1120_QHQEC,coupon資訊_Yes,0.0,-0.34,推薦,36.974946,7.763381,44.738326,29.85,1335.439043
7,1120_QHQEC,EDM_Yes,0.0,-0.31,推薦,36.974946,6.898889,43.873835,29.85,1309.633974
8,1120_QHQEC,產品搭售B資訊_Yes,0.0,-0.25,推薦,36.974946,5.338435,42.313381,29.85,1263.054423
0,1120_QHQEC,加值功能C_Yes,1.0,0.20,轉換,36.974946,3.458271,40.433217,29.85,1206.931513


## 六、存留表

*存留表：將資料集時間因素考慮入模型，可以快速知道每一位會員每一個時間點存留的可能性*

In [27]:
customer_survival_status = cph.predict_expectation(customer_data)

In [28]:
customer_survival_status

1120_QHQEC    36.974946
1111_CNQDE    61.135099
4668_QPYBI    39.603517
1121_CFTCW    67.884256
2241_HQITU     9.028562
                ...    
6840_RESQB    69.026911
2244_XADUH    68.253306
4801_JZAZL    37.955346
8461_LTMID    27.165890
4186_AJIEI    70.598521
Length: 7043, dtype: float64

**reset index，將customer id 給變成可調整的欄位</font>**

In [29]:
customer_survival_status = customer_survival_status.reset_index()
customer_survival_status.columns = ['customerID', '個體預測存留月數']
customer_survival_status = customer_survival_status.sort_values('個體預測存留月數')
customer_survival_status

,customerID,個體預測存留月數
3064,1811_DIWPT,8.847437
6764,1660_HDPJQ,8.895777
6491,2128_FTTQZ,8.895777
6488,0488_CSLFR,8.922223
4081,1661_TTALD,8.931051
...,...,...
632,1111_IWJIQ,71.860994
2983,4614_HCNUA,71.861049
2372,0140_BCQCF,71.861494
4513,8180_QQLTC,71.861918


In [30]:
customer_survival_status.to_csv('存留表.csv', encoding = 'UTF-8-sig')


**產出10月的「行銷策略的調整表」</font>**<br>
**我只要抓customer_dying`['customerID']`的前10個</font>**

In [31]:
customer_dying = customer_survival_status[customer_survival_status['個體預測存留月數'] <= 10 ]
for i in customer_dying['customerID'][0:10]:
    customer_marketng_strategy_func(customer_data=customer_data,
                            cph=cph,
                            customer_test_name = i,
                            keep_coef2 = keep_coef2)

c:\Users\saman\OneDrive\桌面\TMR\課程\機器學習實作\day3_課後\機器學習實作案例\2_顧客流失預警分析\2_顧客流失預警\util.py:207: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\saman\OneDrive\桌面\TMR\課程\機器學習實作\day3_課後\機器學習實作案例\2_顧客流失預警分析\2_顧客流失預警\util.py:208: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\saman\OneDrive\桌面\TMR\課程\機器學習實作\day3_課後\機器學習實作案例\2_顧客流失預警分析\2_顧客流失預警\util.py:209: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [32]:
from util import move_file
move_file(dectect_name = '顧客終身價值', folder_name='10months')